In [ ]:
import PIL
import torch
import kornia
import numpy as np
from diffusers.pipelines import AutoPipelineForText2Image

In [ ]:
pipeline = AutoPipelineForText2Image.from_pretrained(
        "/root/autodl-tmp/stable-diffusion/sdxl-turbo",
        torch_dtype=torch.float32)

In [ ]:
image = PIL.Image.open('/root/autodl-tmp/fsc147-controlnet/image/3.png')
density = PIL.Image.open('/root/autodl-tmp/fsc147-controlnet/density/3.png')
from clip_count.run import Model
model =  Model.load_from_checkpoint("/root/autodl-tmp/clip-count/clipcount_pretrained.ckpt", strict=False).cuda()
model.eval()

from diffusers.image_processor import VaeImageProcessor
image_processor = VaeImageProcessor(vae_scale_factor=8)
img_tensor = image_processor.preprocess(image).to('cuda')

def transform_img_tensor(image):
    """
    Transforms an image based on the specified classifier input configurations.
    """
    # image = kornia.geometry.transform.resize(image, 224, interpolation="bicubic")
    image = kornia.geometry.transform.resize(image, 224)
    image = kornia.geometry.transform.center_crop(image, (224, 224))
    # image = T.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD)(image)
    return image

img_tensor = transform_img_tensor(img_tensor)
with torch.cuda.amp.autocast():
    orig_output = model(img_tensor, "A photo of 11 balls")


In [ ]:
orig_output.max(), orig_output.min(), orig_output.mean(), orig_output.std(),orig_output.sum()

In [ ]:
pred_cnt = (torch.sum(orig_output)/60).item()
pred_cnt
